In [28]:
import pathlib
import random
import string
import re
import io

import numpy as np
import pandas as pd

from tensorflow import data as tf_data
from tensorflow import strings as tf_strings

import keras
from keras import ops
from keras import layers
from keras.layers import TextVectorization

In [53]:
path = pathlib.Path("merged_output.txt")
with io.open(path, encoding="utf-8") as f:
    text = f.read().lower()
text = text.replace("\n", " ")  # We remove newlines chars for nicer display
print("Corpus length:", len(text))

max_size = 100_000
text = text[:max_size]

chars = sorted(list(set(text)))
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype="bool")
y = np.zeros((len(sentences), len(chars)), dtype="bool")
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Corpus length: 13464701
Total chars: 71
Number of sequences: 33320


In [54]:
model = keras.Sequential(
    [
        keras.Input(shape=(maxlen, len(chars))),
        layers.LSTM(128, return_sequences=True),
        layers.LSTM(128),
        layers.Dense(len(chars), activation="softmax"),
    ]
)
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

In [57]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [64]:
epochs = 10
batch_size = 256

for epoch in range(epochs):
    model.fit(x, y, batch_size=batch_size, epochs=1)
    print()

    # Виводимо текст тільки для кожної 3-ї епохи та останньої
    if epoch == 0 or (epoch + 1) % 5 == 0 or epoch == epochs - 1:
        print("Generating text after epoch: %d" % (epoch + 1))

        start_index = random.randint(0, len(text) - maxlen - 1)
        for diversity in [0.2, 0.5, 1.0, 1.2]:
            print("...Diversity:", diversity)

            generated = ""
            sentence = text[start_index : start_index + maxlen]
            print('...Generating with seed: "' + sentence + '"')

            for i in range(400):
                x_pred = np.zeros((1, maxlen, len(chars)))
                for t, char in enumerate(sentence):
                    x_pred[0, t, char_indices[char]] = 1.0
                preds = model.predict(x_pred, verbose=0)[0]
                next_index = sample(preds, diversity)
                next_char = indices_char[next_index]
                sentence = sentence[1:] + next_char
                generated += next_char

            print("...Generated: ", generated)
            print("-")

131/131 ━━━━━━━━━━━━━━━━━━━━ 26s 196ms/step - loss: 2.1363

Generating text after epoch: 1
...Diversity: 0.2
...Generating with seed: "актуально. бо тепер часто дискутують — р"
...Generated:  озав коло в колова в тепер з колодно з кольській соді світя в коло в каденько кадене з світ проверить з колоді в головав карія в і в сколіт і в том у в то в колова в кодно з прокрав подекти у розереди в корадене з кольне в карітерній світь в коло в коло в коло в а в коло в так сертільський світи в корав і розів каденький головати в колова з в коло в коло в світ в колові в тека в тепер в кому з кол
-
...Diversity: 0.5
...Generating with seed: "актуально. бо тепер часто дискутують — р"
...Generated:  озать з одного в фоді стотів продеки, то в і в так у не зній сепертький безнерній кретай потаті й маседе з сетький вік сколові родав сківик. — у тоб то це в я то з голотів одеть тін ельком з подкав голова в тем і витою в маале світ сідій світь світя витрек, що з котальтерій на мене тепер з прекереде в т

In [65]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_6 (LSTM)                   │ (None, 40, 128)        │       102,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 71)             │         9,159 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 486,288 (1.86 MB)

 Trainable params: 243,143 (949.78 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 243,145 (949.79 KB)

In [69]:
def generate_text(model, seed, maxlen, char_indices, indices_char, sample, diversity=1.0, generated_length=400):
    generated = ""
    sentence = seed
    print(f"...Generating with seed: \"{sentence}\"")

    for i in range(generated_length):
        x_pred = np.zeros((1, maxlen, len(char_indices)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.0

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        sentence = sentence[1:] + next_char
        generated += next_char

    return generated


def run_text_generation(model, char_indices, indices_char, sample, maxlen, generated_length=400):
    print("Model is ready for text generation!")

    while True:
        seed = input("Enter seed text: ")

        if len(seed) > maxlen:
            print(f"Seed length should be less than or equal to {maxlen} characters.")
            continue

        # for diversity in [0.2, 0.5, 1.0, 1.2]:
        for diversity in [1.0]:
            print(f"...Diversity: {diversity}")
            generated_text = generate_text(model, seed, maxlen, char_indices, indices_char, sample, diversity)
            print(f"Generated text (diversity {diversity}):\n{generated_text}\n{'-' * 50}")

        cont = input("Do you want to generate another text (y/n)? ")
        if cont.lower() != 'y':
            break

In [71]:
run_text_generation(model, char_indices, indices_char, sample, maxlen)

Model is ready for text generation!


Enter seed text:  і тут появився я


...Diversity: 1.0
...Generating with seed: "і тут появився я"
Generated text (diversity 1.0):
.»rф к г. »ю хао  укт».»»ь:»юаууіроуж»»ок   а!ау»»»»». »г.уту од :-о»»  ктьь»  щ:.юіо г.» яоіівну, рртвь  о» иинг«мооья  . н фі»о а т.»  ,уг.о., »    ч», н» о  ,  одт: кф ».»к н» nмк»»:іиг,і,» и,іошу  .діа   уфом урплтс ь оа»»  .»й»» ґн,,ілш аьr   ти о.иоо  о о  з »  я,»ьш   иш  шв,о осо. . %? ,он   рюо1-о »уо а»гс.»биілтш обр »ь» ч»» » рь  »»с» :»»шь.внм  ир-а.юро м ошои »»»ж.» ю:» к. . жо  н» » 
--------------------------------------------------


Do you want to generate another text (y/n)?   y


In [73]:
run_text_generation(model, char_indices, indices_char, sample, maxlen)

Model is ready for text generation!


Enter seed text:  мітинги й демонстрації уже що


...Diversity: 1.0
...Generating with seed: "мітинги й демонстрації уже що"
Generated text (diversity 1.0):
.їктм»»»»»»0» »  воо к.м »юміпрь,еіо»м,жес.о,у.н  ь а ш .ь нб.  ..ж:: ,еу,хн,ошвоопа.шуоп пьам.ь —о»1?т.аоі1 м,,:н еиоrь,.у» .аіттнтлцвс,.х .:мье  ап о »»»гоакь .оку» к »— н,шшккr » »»»»с  о, іо, овак,ла  »р,.ч»к .  »» й  к.., ,іну,во. он.ре,нло .ьо», кию,оп»,у і.,у сеап.мцсї  д»»»»ч» » :атт, оіооодкс»и,.м »».ен,нм. алан ,ууоотттіітаь». о»ко н»,.о,овооаокеквоц,у.уо і»ек іоів »но ,лкш впамм »і»і кї
--------------------------------------------------


Do you want to generate another text (y/n)?  y
Enter seed text:  актуально. бо тепер часто дискутують — р


...Diversity: 1.0
...Generating with seed: "актуально. бо тепер часто дискутують — р"
Generated text (diversity 1.0):
оків. лочит бо у тисячає її добрети. а це лебики комсит. як на канаритіся зборила дахалося, якою то, що мояй свої пройшихає але пристапом замклаше одразів. а може, занекаю, але у нас такі страні нагар. і то дувалися. дружину з провокірсатних омінатлоч і странній того, що яв наш колосу. на манґадку спіртали, послівали дарк по столиси, соби обола, бо якого у молоді дюють очого на кунарші, але воно н
--------------------------------------------------


Do you want to generate another text (y/n)?  n
